In [ ]:
from selenium import webdriver
from lxml import etree
import requests
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [ ]:
def get_web_data(dom=None, cookies=None):
    '''
    获取每页评论数据
    '''
    names = dom.xpath('//div[@class="comment-item "]//span[@class="comment-info"]/a/text()')           # 用户名
    ratings = dom.xpath('//div[@class="comment-item "]//span[@class="comment-info"]/span[2]/@class')   # 评分
    times = dom.xpath('//div[@class="comment-item "]//span[@class="comment-info"]/span[@class="comment-time "]/@title')     # 评论发布时间
    message = dom.xpath('//div[@class="comment-item "]//div[@class="comment"]//span[@class="short"]/text()')    # 短评正文
    user_url = dom.xpath('//div[@class="comment-item "]//span[@class="comment-info"]/a/@href')           # 用户主页网址
    votes = dom.xpath('//div[@class="comment-item "]//div[@class="comment"]//span[@class="votes"]/text()')    # 赞同数量
    cities = []
    load_times = []
    for i in user_url:
        web_data = requests.get(i, cookies=cookies)
        dom_url = etree.HTML(web_data.text, etree.HTMLParser(encoding='utf-8'))
        address = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/a/text()')                    # 用户居住地
        load_time = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/div[@class="pl"]/text()')   # 用户入会时间
        cities.append(address)
        load_times.append(load_time)
        time.sleep(3)
    ratings = ['' if 'rating' not in i else int(re.findall('\d{2}', i)[0]) for i in ratings]   # 评分数据整理
    load_times = ['' if i == [] else i[1].strip()[:-2] for i in load_times]   # 入会数据整理
    cities = ['' if i == [] else i[0] for i in cities]   # 居住地数据整理
    data = pd.DataFrame({
        '用户名': names,
        '居住城市': cities,
        '加入时间': load_times,
        '评分': ratings,
        '发表时间': times,
        '短评正文': message,
        '赞同数量': votes
    })
    return data


In [ ]:
driver = webdriver.Chrome()   # 启动chrome浏览器
url = 'https://movie.douban.com/subject/26266893/comments?status=P'
driver.get(url)   # 获取网页源码数据

In [ ]:
cookies_str = 'bid=4oKfJYlCtxM; ap_v=0,6.0; _pk_ses.100001.4cf6=*; __utma=30149280.2037271657.1609905129.1609905129.1609905129.1; __utmc=30149280; __utmz=30149280.1609905129.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utma=223695111.99416835.1609905129.1609905129.1609905129.1; __utmb=223695111.0.10.1609905129; __utmc=223695111; __utmz=223695111.1609905129.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); dbcl2="146466491:ZtOBgFHlFds"; ck=X8-j; push_noty_num=0; push_doumail_num=0; __utmt=1; __utmv=30149280.14646; __utmb=30149280.2.10.1609905129; _pk_id.100001.4cf6=6bf19fc7f349fb52.1609905129.1.1609905561.1609905129.'
cookies = {}
for i in cookies_str.split(';'):  # 将cookies整理成所需格式
    k, v = i.split('=', 1)
    cookies[k] = v

In [ ]:
all_data = pd.DataFrame()
wait = WebDriverWait(driver, 10)
while True:
    wait.until(
        EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '#comments > div:nth-child(20) > div.comment > h3 > span.comment-info > a') # 当前页最后一个“用户”按钮是否可以点击
    )
    )
    dom = etree.HTML(driver.page_source, etree.HTMLParser(encoding='utf-8'))
    data = get_web_data(dom=dom, cookies=cookies)
    all_data = pd.concat([all_data, data], axis=0)
    
    if driver.find_element_by_css_selector('#paginator > a.next')==[]:   # 判定是否还有“后页”按钮
        break
    
    confirm_bnt = wait.until(
    EC.element_to_be_clickable(
    (By.CSS_SELECTOR, '#paginator > a.next')
    )
    )
    # 找到该元素
    
    confirm_bnt.click() # 点击页面跳转

In [ ]:
all_data.tocsv('douban.csv', index=None, encoding='GB18030')